<a href="https://colab.research.google.com/github/irravasyuk/SQL_2/blob/master/%D0%94%D0%97_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [3]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [4]:
df = df[['Rent', 'Size', 'Furnishing Status', 'City']]
df.head()

,Rent,Size,Furnishing Status,City
0,10000.0,1100.0,Unfurnished,Kolkata
1,20000.0,800.0,Semi-Furnished,Kolkata
2,17000.0,1000.0,Semi-Furnished,Kolkata
3,10000.0,800.0,Unfurnished,Kolkata
4,7500.0,850.0,Unfurnished,Kolkata


In [15]:
df = df.dropna(subset=['Rent'])

# Завдання 1
Ввиведіть основні числові характеристики даних, розділіть на тренувальну та тестову чатини

In [16]:
df.describe()

,Rent,Size
count,4.369000e+03,4050.000000
mean,3.461306e+04,967.004444
std,7.740818e+04,630.252303
min,1.200000e+03,10.000000
25%,1.000000e+04,550.000000
50%,1.600000e+04,850.000000
75%,3.300000e+04,1200.000000
max,3.500000e+06,7000.000000


In [17]:
df.isnull().sum()

Rent                   0
Size                 319
Furnishing Status    320
City                 324
dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

X = df.drop('Rent', axis=1)
y = df['Rent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X

,Size,Furnishing Status,City
0,1100.0,Unfurnished,Kolkata
1,800.0,Semi-Furnished,Kolkata
2,1000.0,Semi-Furnished,Kolkata
3,800.0,Unfurnished,Kolkata
4,850.0,Unfurnished,Kolkata
...,...,...,...
4741,1000.0,Semi-Furnished,Hyderabad
4742,2000.0,Semi-Furnished,Hyderabad
4743,1750.0,Semi-Furnished,Hyderabad
4744,1500.0,Semi-Furnished,Hyderabad


# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [21]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [22]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('num', num_pipeline, ['Size']),
    ('cat', cat_pipeline, ['Furnishing Status', 'City'])
])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


# Завдання 6
Виведіть основні метрики моделі

In [24]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Size']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Furnishing Status',
                                                   'City'])])),
                ('regressor', LinearRegression())])

In [27]:
y_pred = model.predict(X_test)

In [28]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [29]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, model.predict(X_test))

all_metrics

,train,test
MAE,24574.18,23456.96
RMSE,72071.11,42598.22
R2,0.23,0.36


# Завдання 7
Збережіть модель, завантажте її та спрогнозуйте ціну автомобіля

In [31]:
import joblib

joblib.dump(model, 'model.pkl')

loaded_model = joblib.load('model.pkl')

data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

data


,Size,Furnishing Status,City
0,900,Semi-Furnished,Mumbai


In [32]:
predicted_rent = loaded_model.predict(data)
print(f'Predicted Rent: {predicted_rent[0]}')

Predicted Rent: 68737.78367649853
